In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import itertools
import re
from itertools import product
import operator
from functools import reduce
import polars as pl
#from pandas import option_context
#%pip install openpyxl
%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn-pastel')
plt.style.use('seaborn-dark')

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_94778/2875002457.py:15: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-pastel')
/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_94778/2875002457.py:16: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


In [1]:
import matplotlib
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
matplotlib.style.use('dark_background')
color_list = ['white','skyblue','pink', 'tomato', 'slateblue', 'wheat', 'lightgrey', 'limegreen', 'royalblue', 'yellowgreen', 'lightseagreen', 'orchid', 'orange']
cmap = LinearSegmentedColormap.from_list("",color_list)

In [2]:
# Import all the google connexion packages
import os
import google.auth
from google.cloud import bigquery
from google.auth import impersonated_credentials

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/a20015173/.config/gcloud/legacy_credentials/ekaterina.rezanovich@adeo.com/adc.json"
client_1 = bigquery.Client(project = 'ccdp-transverse-data-dev')

NPS DATA

In [3]:
# questions data
filepath_1 = "./datasets/questions.parquet"
if not os.path.isfile(filepath_1):

    query = """
    select t1.survey_id, t1.name , t2.question_id, t2.name, t2.content
    from `dfdp-customer-exp-mt-prod.customer_experience_measure_bu001.client_experience_survey` as t1
    left join `dfdp-customer-exp-mt-prod.customer_experience_measure_bu001.client_experience_question` as t2 on t2.survey_id = t1.survey_id
    where t2.question_id in ('SV_81jKyfOtepO6Weq__QID1_NPS_GROUP', 'SV_4OUB8z9FwEgBiU6__QID1_NPS_GROUP', 'SV_5nFeSHRTsx1yxWC__QID1_NPS_GROUP', 'SV_77FLHcnyIyMM07Q__QID1_NPS_GROUP', 'SV_bj9J6UgbM2yIATQ__QID13_NPS_GROUP',
    'SV_2fA21joHLvPEZoi__QID1_NPS_GROUP', 'SV_5nVqXD824PZXj14__QID1_NPS_GROUP')"""

    dl_data = pd.read_gbq(query=query, project_id="ccdp-transverse-data-dev")
    dl_data.to_parquet(filepath_1)

df_q = pd.read_parquet(filepath_1)

In [4]:
# answers data
filepath_2 = "./datasets/answers.parquet"
if not os.path.isfile(filepath_2):

    query = """
select t3.answer_id, t3.feedback_id, t3.question_id, t3.score, t3.score_category, t3.comment,
t4.touchpoint_id, t4.store_code, t4.user_id, t4.user_id_type, cast(t4.date as date) as answer_date
from `dfdp-customer-exp-mt-prod.customer_experience_measure_bu001.client_experience_answer` as t3
left join `dfdp-customer-exp-mt-prod.customer_experience_measure_bu001.client_experience_feedback` as t4 on t4.feedback_id = t3.feedback_id
where t3.question_id in ('SV_81jKyfOtepO6Weq__QID1_NPS_GROUP', 'SV_4OUB8z9FwEgBiU6__QID1_NPS_GROUP', 'SV_5nFeSHRTsx1yxWC__QID1_NPS_GROUP', 'SV_77FLHcnyIyMM07Q__QID1_NPS_GROUP', 'SV_bj9J6UgbM2yIATQ__QID13_NPS_GROUP',
'SV_2fA21joHLvPEZoi__QID1_NPS_GROUP', 'SV_5nVqXD824PZXj14__QID1_NPS_GROUP')
    """

    dl_data = pd.read_gbq(query=query, project_id="ccdp-transverse-data-dev")
    dl_data.to_parquet(filepath_2)

df_a = pd.read_parquet(filepath_2)

In [5]:
df_nps_all = df_a.merge(df_q, left_on = 'question_id', right_on = 'question_id', how='left')

In [6]:
del df_nps_all['name_1'], df_nps_all['user_id_type'],  df_nps_all['content']

In [7]:
# filter only nps with user_id
df_nps = df_nps_all[df_nps_all['user_id'].notna()]

In [8]:
# first and last TP of the customer
df_nps_dates = df_nps.groupby('user_id')['answer_date'].agg(['min', 'max']).reset_index()
df_nps_dates.columns = ['uid', 'first_nps_date', 'last_nps_date']

In [9]:
df_nps_1 = df_nps.merge(df_nps_dates, left_on='user_id', right_on='uid', how='left')
del df_nps_1['uid']

In [10]:
# nb FB of the customer
df_nps_nb_fb = df_nps.groupby('user_id')['feedback_id'].nunique().reset_index()
df_nps_nb_fb.columns = ['uid', 'nb_fb']

In [11]:
df_nps_2 = df_nps_1.merge(df_nps_nb_fb, left_on='user_id', right_on='uid', how='left')
del df_nps_2['uid']

In [12]:
# 0 for detractor, 1 for passive, 2 for promotor
def fb_type(df):
    if df['score'] <= 6 :
        return 0
    elif df['score'] >= 9:
        return 2
    else:
        return 1

In [13]:
df_nps_2['score'].unique()

<IntegerArray>
[8, 10, 9, 4, 0, 3, 6, 5, 1, 7, 2, <NA>]
Length: 12, dtype: Int64

In [14]:
df_nps_3 = df_nps_2[df_nps_2['score'].notna()]

In [15]:
df_nps_3['fb_type'] = df_nps_3.apply(fb_type, axis=1)

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_94778/761178500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nps_3['fb_type'] = df_nps_3.apply(fb_type, axis=1)


In [16]:
# types of the customer fb
df_nps_fb_typ = df_nps_3.pivot_table(index='user_id', columns='fb_type', values='feedback_id', aggfunc='nunique').reset_index()
df_nps_fb_typ.columns = ['uid', '0', '1', '2']
df_nps_fb_typ = df_nps_fb_typ.fillna(0)
df_nps_fb_typ.head()

,uid,0,1,2
0,1000000,0.0,0.0,2.0
1,10000024,1.0,0.0,0.0
2,1000004,0.0,0.0,1.0
3,1000011,2.0,0.0,0.0
4,10000188,0.0,0.0,1.0


In [17]:
def fb_type_res(df):
    if df['0']==0 and df['1']==0:
        return "only positive"
    elif df['0']==0 and df['2']==0:
        return "only passive"
    elif df['1']==0 and df['2']==0:
        return "only negative"
    elif df['0']==0:
        return "passive and positive"
    elif df['1']==0:
        return "negative and positive"  
    elif df['2']==0:
        return "negative and passive"  
    else:
        return "negative, passive, positive"

In [18]:
df_nps_fb_typ['fb_type'] = df_nps_fb_typ.apply(fb_type_res, axis=1)

In [19]:
def fb_type_res_nb(df):
    if (df['0']+df['1']+df['2']) > 1:
        return "mutli"
    else:
        return "mono"

In [20]:
df_nps_fb_typ['fb_nb_type'] = df_nps_fb_typ.apply(fb_type_res_nb, axis=1)

In [21]:
df_nps_fb_typ.columns = ['uid', 'nb_fb_neg', 'nb_fb_pas', 'nb_fb_pos', 'fb_group', 'fb_nb_type']

In [22]:
df_nps_4 = df_nps_3.merge(df_nps_fb_typ, left_on='user_id', right_on='uid', how='left')
del df_nps_4['uid']

In [23]:
df_nps_4.head(2)

,answer_id,feedback_id,question_id,score,score_category,comment,touchpoint_id,store_code,user_id,answer_date,...,name,first_nps_date,last_nps_date,nb_fb,fb_type,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type
0,R_111w3BVzUYwknmY__QID1_NPS_GROUP,R_111w3BVzUYwknmY,SV_81jKyfOtepO6Weq__QID1_NPS_GROUP,8,Passif,"Bonjour, \nJe suis entièrement satisfait de ...",SV_81jKyfOtepO6Weq,<NA>,10217876,2023-08-20,...,LMFR - Client SAV - PROD,2022-04-09,2023-08-20,14,1,0.0,1.0,13.0,passive and positive,mutli
1,R_3htf0IgZ0ecDb8u__QID1_NPS_GROUP,R_3htf0IgZ0ecDb8u,SV_81jKyfOtepO6Weq__QID1_NPS_GROUP,10,Promoteur,None,SV_81jKyfOtepO6Weq,<NA>,54974624,2023-08-16,...,LMFR - Client SAV - PROD,2023-08-16,2023-08-16,1,2,0.0,0.0,1.0,only positive,mono


In [24]:
df_nps_summary_type = df_nps_4.groupby('fb_nb_type')['user_id'].nunique().reset_index()
df_nps_summary_type['share_%'] = (df_nps_summary_type['user_id'] / df_nps_summary_type['user_id'].sum())*100
df_nps_summary_type

,fb_nb_type,user_id,share_%
0,mono,889117,72.614886
1,mutli,335311,27.385114


In [25]:
df_nps_summary_group = df_nps_4.groupby('fb_group')['user_id'].nunique().reset_index()
df_nps_summary_group['share_%'] = (df_nps_summary_group['user_id'] / df_nps_summary_group['user_id'].sum())*100
df_nps_summary_group = df_nps_summary_group.sort_values(by='share_%', ascending=False)
df_nps_summary_group

,fb_group,user_id,share_%
5,only positive,680303,55.560882
4,only passive,277548,22.667564
3,only negative,129318,10.561503
6,passive and positive,77828,6.356274
1,negative and positive,26855,2.193269
0,negative and passive,21942,1.792020
2,"negative, passive, positive",10634,0.868487


In [26]:
df_nps_summary = df_nps_4.groupby(['fb_nb_type', 'fb_group'])['user_id'].nunique().reset_index()
df_nps_summary['share_%'] = (df_nps_summary['user_id'] / df_nps_summary['user_id'].sum())*100
df_nps_summary = df_nps_summary.sort_values(by=['fb_nb_type','share_%'], ascending=[True, False])
df_nps_summary

,fb_nb_type,fb_group,user_id,share_%
2,mono,only positive,524428,42.830448
1,mono,only passive,246769,20.153819
0,mono,only negative,117920,9.630619
8,mutli,only positive,155875,12.730434
9,mutli,passive and positive,77828,6.356274
7,mutli,only passive,30779,2.513745
4,mutli,negative and positive,26855,2.193269
3,mutli,negative and passive,21942,1.792020
6,mutli,only negative,11398,0.930884
5,mutli,"negative, passive, positive",10634,0.868487


In [27]:
# filter nps data for clear user_id
no_user_id = ['undefined', 'a', 'ra']
df_nps = df_nps.query('user_id not in @no_user_id')
df_nps['user_id'] = df_nps['user_id'].astype('int')

In [28]:
df_nps_users = df_nps.groupby('user_id')['feedback_id'].nunique().reset_index()
nps_users = df_nps_users['user_id'].to_list()

SALES DATA

!!! Read df_s_6

In [56]:
df_s0 = pd.read_parquet('./datasets/sales_gr/df_s_3p.parquet')
df_s1 = pd.read_parquet('./datasets/sales_gr/df_s_22_1_1p.parquet')

In [28]:
df_s2 = pd.read_parquet('./datasets/sales_gr/df_s_22_2_1p.parquet')

In [29]:
df_s3 = pd.read_parquet('./datasets/sales_gr/df_s_22_3_1p.parquet')

In [30]:
df_s4 = pd.read_parquet('./datasets/sales_gr/df_s_22_4_1p.parquet')

In [31]:
df_s5 = pd.read_parquet('./datasets/sales_gr/df_s_23_1_1p.parquet')

In [32]:
df_s6 = pd.read_parquet('./datasets/sales_gr/df_s_23_2_1p.parquet')

In [33]:
df_s7 = pd.read_parquet('./datasets/sales_gr/df_s_23_3_1p.parquet')

In [57]:
df_s_1p = df_s0.append([df_s1,df_s2,df_s3])

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_73845/2518559154.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_s_1p = df_s0.append([df_s1,df_s2,df_s3])


In [35]:
df_s_2p = df_s4.append([df_s5,df_s6,df_s7])

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_73845/3628812971.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_s_2p = df_s4.append([df_s5,df_s6,df_s7])


In [58]:
df_s = df_s_1p.append(df_s_2p)

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_73845/4204960968.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_s = df_s_1p.append(df_s_2p)


In [59]:
df_s.head()

,store_id,cart_date,cart_id,user_id,top_l_c,sales_channel,delivery_channel,amount_to,qty
0,380,2022-01-01,001-20350L606-A,56210348,0,Website,Delivery,23.40,1
1,380,2022-01-01,001-21023L763-A,77795876,0,Website,Delivery,42.90,1
2,380,2022-01-01,001-21080L164-A,67739822,0,Website,Delivery,104.99,1
3,380,2022-01-01,001-21089L571-A,70027761,0,Website,Delivery,25.45,1
4,380,2022-01-01,001-21209L741-A,64195109,0,Website,Delivery,27.99,1


In [38]:
df_s['cart_date'].min()

datetime.date(2022, 1, 1)

In [51]:
df_s['cart_date'].max()

datetime.date(2023, 10, 19)

In [40]:
df_s['store_id'].unique()

<IntegerArray>
[  3,   4,   5,   6,  10,  11,  12,  13,  14,  15,
 ...
 291, 292, 293, 294, 295, 296, 297, 298, 380, 230]
Length: 146, dtype: Int64

In [41]:
df_s['sales_channel'].unique()

array(['Store', 'Website'], dtype=object)

In [42]:
df_s['delivery_channel'].unique()

array(['Self-service', 'Pickup', 'Self-service - Sales Bulletin',
       'Delivery'], dtype=object)

In [60]:
df_s['user_id'].nunique()

9567696

In [62]:
# filter users with nps reviews
df_s_nps = df_s.query('user_id in @nps_users')

In [65]:
print("Nb customers with purchase", df_s['user_id'].nunique())
print("Nb customers with purchase and nps", df_s_nps['user_id'].nunique())

Nb customers with purchase 9567696
Nb customers with purchase and nps 1216062


In [67]:
1216062 / 9567696

0.12710081925679914

In [68]:
# first,last dates of purchases by channel
df_s_nps_ch_dates = df_s_nps.pivot_table(index='user_id', columns = 'sales_channel',values='cart_date',\
                                          aggfunc=['min','max']).reset_index()
df_s_nps_ch_dates.columns = ['uid', 'min_date_store', 'min_date_web', 'max_date_store', 'max_date_web']
df_s_nps_ch_dates.head(2)

,uid,min_date_store,min_date_web,max_date_store,max_date_web
0,14,2022-02-12,2022-03-08,2023-09-05,2023-07-15
1,15,2022-02-09,2022-01-03,2023-10-07,2023-10-10


In [72]:
# first,last dates of purchases in general
df_s_nps_dates = df_s_nps.groupby('user_id').agg({"cart_date": ['min', 'max']}).reset_index()
df_s_nps_dates.columns = ['uid', 'min_date', 'max_date']
df_s_nps_dates.head(2)

,uid,min_date,max_date
0,14,2022-02-12,2023-09-05
1,15,2022-01-03,2023-10-10


In [73]:
#merge this info to sales table
df_s_2 = df_s_nps.merge(df_s_nps_ch_dates, left_on='user_id', right_on='uid', how='left')
del df_s_2['uid']
df_s_2.head(2)

,store_id,cart_date,cart_id,user_id,top_l_c,sales_channel,delivery_channel,amount_to,qty,min_date_store,min_date_web,max_date_store,max_date_web
0,380,2022-01-01,001-21209L741-A,64195109,0,Website,Delivery,27.99,1,2023-08-11,2022-01-01,2023-08-11,2023-10-08
1,380,2022-01-01,001-21222L1732-A,55072654,0,Website,Delivery,249.00,1,2022-01-05,2022-01-01,2022-05-13,2023-07-20


In [74]:
df_s_3 = df_s_2.merge(df_s_nps_dates, left_on='user_id', right_on='uid', how='left')
del df_s_3['uid']
df_s_3.head(2)

,store_id,cart_date,cart_id,user_id,top_l_c,sales_channel,delivery_channel,amount_to,qty,min_date_store,min_date_web,max_date_store,max_date_web,min_date,max_date
0,380,2022-01-01,001-21209L741-A,64195109,0,Website,Delivery,27.99,1,2023-08-11,2022-01-01,2023-08-11,2023-10-08,2022-01-01,2023-10-08
1,380,2022-01-01,001-21222L1732-A,55072654,0,Website,Delivery,249.00,1,2022-01-05,2022-01-01,2022-05-13,2023-07-20,2022-01-01,2023-07-20


In [75]:
df_nps_5 = df_nps_4.query('user_id not in @no_user_id')
df_nps_5['uid'] = df_nps_5['user_id'].astype('int')

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_73845/692595386.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nps_5['uid'] = df_nps_5['user_id'].astype('int')


In [76]:
df_nps_6 = df_nps_5.groupby(['uid', 'first_nps_date', 'last_nps_date', 'nb_fb_neg', 'nb_fb_pas','nb_fb_pos','fb_group',\
                                 'fb_nb_type'])['nb_fb'].mean().reset_index()

In [77]:
df_nps_6.head()

,uid,first_nps_date,last_nps_date,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type,nb_fb
0,14,2022-05-04,2022-05-04,0.0,1.0,0.0,only passive,mono,1.0
1,15,2022-11-13,2023-01-22,0.0,0.0,2.0,only positive,mutli,2.0
2,16,2023-02-07,2023-02-07,0.0,0.0,1.0,only positive,mono,1.0
3,19,2022-06-29,2022-11-15,0.0,1.0,1.0,passive and positive,mutli,2.0
4,22,2022-07-05,2023-07-19,0.0,2.0,1.0,passive and positive,mutli,3.0


In [79]:
df_s_3['cart_id'].nunique()

25961354

In [85]:
# users' purchases characterisrics
df_s_ch = df_s_3.pivot_table(index='user_id', columns='sales_channel', 
                             aggfunc={'amount_to' : 'sum', 'qty' : 'sum', 'cart_id':'nunique'}).reset_index()

In [86]:
df_s_ch.columns = ['user_id','amount_to_store','amount_to_web','nb_carts_store', 'nb_carts_web', 'qty_store','qty_web']
df_s_ch.head(2)

,user_id,amount_to_store,amount_to_web,nb_carts_store,nb_carts_web,qty_store,qty_web
0,14,315.51,341.01,20.0,4.0,34.0,9.0
1,15,5968.82,1072.00,67.0,14.0,273.0,82.0


In [87]:
df_s_nps_ch_dates.head(2)

,uid,min_date_store,min_date_web,max_date_store,max_date_web
0,14,2022-02-12,2022-03-08,2023-09-05,2023-07-15
1,15,2022-02-09,2022-01-03,2023-10-07,2023-10-10


In [88]:
df_s_nps_dates.head(2)

,uid,min_date,max_date
0,14,2022-02-12,2023-09-05
1,15,2022-01-03,2023-10-10


In [89]:
df_nps_6.head(2)

,uid,first_nps_date,last_nps_date,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type,nb_fb
0,14,2022-05-04,2022-05-04,0.0,1.0,0.0,only passive,mono,1.0
1,15,2022-11-13,2023-01-22,0.0,0.0,2.0,only positive,mutli,2.0


In [90]:
df_s_4 = df_s_ch.merge(df_s_nps_ch_dates, left_on='user_id', right_on='uid', how='left')

In [91]:
df_s_5 = df_s_4.merge(df_s_nps_dates, left_on='user_id', right_on='uid', how='left')

In [156]:
df_s_6 = df_s_5.merge(df_nps_6, left_on='user_id', right_on='uid', how='left')

In [150]:
del df_s_6['uid_x'], df_s_6['uid_y'], df_s_6['uid']

In [152]:
df_s_6.to_parquet('./datasets/sales_summary_nps.parquet')

In [ ]:
print("Nb customers with purchase", df_s['user_id'].nunique())
print("Nb customers with purchase and nps", df_s_6['user_id'].nunique())

In [98]:
print('% of customers with purchases and NPS: ', (df_s_6['user_id'].nunique()/df_s['user_id'].nunique())*100)

% of customers with purchases and NPS:  12.710081925679914


!!! Read df_s_6

In [29]:
df_s_6 = pd.read_parquet('./datasets/sales_summary_nps.parquet')

In [31]:
print("Nb customers with purchase and nps", df_s_6['user_id'].nunique())

Nb customers with purchase and nps 1216062


### 13 % of all customers that did purchases (web, store, both) gave NPS

In [32]:
df_s_6[df_s_6['user_id']==96972383]

,user_id,amount_to_store,amount_to_web,nb_carts_store,nb_carts_web,qty_store,qty_web,min_date_store,min_date_web,max_date_store,...,min_date,max_date,first_nps_date,last_nps_date,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type,nb_fb
1216057,96972383,NaN,61.85,NaN,1.0,NaN,5.0,NaT,2023-09-27,NaT,...,2023-09-27,2023-09-27,2023-09-16,2023-09-16,0.0,1.0,0.0,only passive,mono,1.0


In [33]:
#quality check store
def q_check(df):
    #ordinary cases
    if ((~pd.isna(df['nb_carts_store'])) and (~pd.isna(df['amount_to_store'])) and (~pd.isna(df['qty_store']))) or \
        (pd.isna(df['nb_carts_store']) and pd.isna(df['amount_to_store']) and pd.isna(df['qty_store'])) or \
        ((~pd.isna(df['nb_carts_web'])) and (~pd.isna(df['amount_to_web'])) and (~pd.isna(df['qty_web']))) or \
        (pd.isna(df['nb_carts_web']) and pd.isna(df['amount_to_web']) and pd.isna(df['qty_web'])) or \
        (((~pd.isna(df['nb_carts_store'])) and (~pd.isna(df['amount_to_store'])) and (~pd.isna(df['qty_store']))) and \
        ((~pd.isna(df['nb_carts_web'])) and (~pd.isna(df['amount_to_web'])) and (~pd.isna(df['qty_web'])))):
        return 1
    # return the items cases
    elif ((~pd.isna(df['nb_carts_store'])) and pd.isna(df['amount_to_store']) and pd.isna(df['qty_store'])) or\
         ((~pd.isna(df['nb_carts_web'])) and pd.isna(df['amount_to_web']) and pd.isna(df['qty_web'])):
        return 2
    else:
        return 0

Note: carts with both purchases anf return, do not take into account amount and qty for this part of the analysis

In [34]:
df_s_6['q_ch'] = df_s_6.apply(q_check, axis=1)

In [35]:
def type_purchase(df):
    #customer did only web purchase
    if pd.isnull(df['min_date_store']) and df['min_date_web'] is not pd.NaT:
        return "only web"
    elif pd.isnull(df['min_date_web']) and df['min_date_store'] is not pd.NaT:
        return "only store"
    elif df['min_date_store'] is not pd.NaT and df['min_date_web'] is not pd.NaT:
        return "store, web"
    else:
        return "?"

In [36]:
df_s_6['purchase_type'] = df_s_6.apply(type_purchase, axis=1)

In [39]:
def last_purchase(df):
    if df['purchase_type'] == "store, web":
        if df['max_date_store'] >= df['max_date_web']:
            return "last store purchase"
        else:
            return "last web purchase" 
    else:
        return df['purchase_type']

In [40]:
df_s_6['last_purchase'] = df_s_6.apply(last_purchase, axis=1)

In [44]:
df_s_6.loc[:,'max_date_store':].head()

,max_date_store,max_date_web,min_date,max_date,first_nps_date,last_nps_date,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type,nb_fb,q_ch,purchase_type,last_purchase
0,2023-09-05,2023-07-15,2022-02-12,2023-09-05,2022-05-04,2022-05-04,0.0,1.0,0.0,only passive,mono,1.0,1,"store, web",last store purchase
1,2023-10-07,2023-10-10,2022-01-03,2023-10-10,2022-11-13,2023-01-22,0.0,0.0,2.0,only positive,mutli,2.0,1,"store, web",last web purchase
2,2023-10-11,2023-02-07,2022-02-14,2023-10-11,2023-02-07,2023-02-07,0.0,0.0,1.0,only positive,mono,1.0,1,"store, web",last store purchase
3,2023-10-19,NaT,2022-05-05,2023-10-19,2022-06-29,2022-11-15,0.0,1.0,1.0,passive and positive,mutli,2.0,1,only store,only store
4,2023-07-20,NaT,2022-01-05,2023-07-20,2022-07-05,2023-07-19,0.0,2.0,1.0,passive and positive,mutli,3.0,1,only store,only store


In [45]:
def return_flag(df):
    # only web customer
    if df['purchase_type'] == 'only web':
        if df['last_nps_date'] >= df['max_date_web']:
            return "no return"
        else:
            return "return"
    # only store customer
    elif df['purchase_type'] == 'only store':
        if df['last_nps_date'] >= df['max_date_store']:
            return "no return"
        else:
            return "return"
    # store, web customer
    elif df['purchase_type'] == 'store, web':
        # last purchase web
        if df['last_purchase'] == 'last web purchase':
            if df['last_nps_date'] >= df['max_date_web']:
                return "no return"
            else:
                if df['last_nps_date'] >= df['max_date_store']:
                    return "return/ store->web"
                else:
                    return "return/ ->store,web"
        #lasr purchase store
        if df['last_purchase'] == 'last store purchase':
            if df['last_nps_date'] >= df['max_date_store']:
                return "no return"
            else:
                if df['last_nps_date'] >= df['max_date_web']:
                    return "return/ web->store"
                else:
                    return "return/ ->web,store"

In [46]:
df_s_6['return_flag'] = df_s_6.apply(return_flag, axis=1)

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_94778/4113624140.py:27: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.
  if df['last_nps_date'] >= df['max_date_store']:
/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_94778/4113624140.py:30: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.
  if df['last_nps_date'] >= df['max_date_web']:
/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_94778/4113624140.py:10: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.
  if df['last_nps_date'] >= df['max_date_store']:
/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_94778/4113624140

In [47]:
df_s_6.to_parquet('./datasets/first_results.parquet')

In [49]:
df_s_6[df_s_6['user_id']==96972383].loc[:,'min_date_store':]

,min_date_store,min_date_web,max_date_store,max_date_web,min_date,max_date,first_nps_date,last_nps_date,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type,nb_fb,q_ch,purchase_type,last_purchase,return_flag
1216057,NaT,2023-09-27,NaT,2023-09-27,2023-09-27,2023-09-27,2023-09-16,2023-09-16,0.0,1.0,0.0,only passive,mono,1.0,1,only web,only web,return


In [66]:
def nb_purchase(df):
    if df['nb_carts_store']>1 or df['nb_carts_web']>1 or (df['nb_carts_store']+df['nb_carts_web'])>1:
        return "multi purchase"
    else:
        return "mono purchase"

In [67]:
df_s_6['purchase_nb'] = df_s_6.apply(nb_purchase, axis=1)

In [69]:
df_s_6[df_s_6['purchase_type']=='only store'].head(10)

,user_id,amount_to_store,amount_to_web,nb_carts_store,nb_carts_web,qty_store,qty_web,min_date_store,min_date_web,max_date_store,...,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type,nb_fb,q_ch,purchase_type,last_purchase,return_flag,purchase_nb
3,19,5674.13,NaN,34.0,NaN,244.0,NaN,2022-05-05,NaT,2023-10-19,...,1.0,1.0,passive and positive,mutli,2.0,1,only store,only store,return,multi purchase
4,22,3663.47,NaN,32.0,NaN,36.0,NaN,2022-01-05,NaT,2023-07-20,...,2.0,1.0,passive and positive,mutli,3.0,1,only store,only store,return,multi purchase
7,57,1070.76,NaN,29.0,NaN,57.0,NaN,2022-01-05,NaT,2023-10-19,...,0.0,2.0,only positive,mutli,2.0,1,only store,only store,return,multi purchase
8,70,1242.50,NaN,27.0,NaN,81.0,NaN,2022-02-14,NaT,2023-10-10,...,0.0,2.0,only positive,mutli,2.0,1,only store,only store,return,multi purchase
11,149,108.90,NaN,2.0,NaN,2.0,NaN,2023-02-27,NaT,2023-03-16,...,0.0,1.0,only positive,mono,1.0,1,only store,only store,no return,multi purchase
14,194,9228.05,NaN,111.0,NaN,679.0,NaN,2022-01-08,NaT,2023-10-12,...,1.0,0.0,only passive,mono,1.0,1,only store,only store,return,multi purchase
15,250,446.00,NaN,9.0,NaN,33.0,NaN,2022-01-13,NaT,2023-08-03,...,0.0,3.0,only positive,mutli,3.0,1,only store,only store,no return,multi purchase
17,284,2475.48,NaN,73.0,NaN,181.0,NaN,2022-01-05,NaT,2023-09-30,...,0.0,4.0,only positive,mutli,4.0,1,only store,only store,return,multi purchase
20,309,581.51,NaN,27.0,NaN,37.0,NaN,2022-01-14,NaT,2023-09-17,...,1.0,0.0,only passive,mono,1.0,1,only store,only store,return,multi purchase
23,322,181.70,NaN,9.0,NaN,16.0,NaN,2022-01-04,NaT,2023-09-16,...,1.0,1.0,"negative, passive, positive",mutli,3.0,1,only store,only store,return,multi purchase


In [54]:
df_s_6['user_id'].nunique()

1216062

In [55]:
df_s_6.shape[0]

1216062

In [76]:
def return_gr(df):
    if df['return_flag']=='no return':
        return 'no return'
    else:
        return "return"

In [77]:
df_s_6['return_gr']= df_s_6.apply(return_gr, axis=1)

In [78]:
df_s_6.head()

,user_id,amount_to_store,amount_to_web,nb_carts_store,nb_carts_web,qty_store,qty_web,min_date_store,min_date_web,max_date_store,...,nb_fb_pos,fb_group,fb_nb_type,nb_fb,q_ch,purchase_type,last_purchase,return_flag,purchase_nb,return_gr
0,14,315.51,341.01,20.0,4.0,34.0,9.0,2022-02-12,2022-03-08,2023-09-05,...,0.0,only passive,mono,1.0,1,"store, web",last store purchase,"return/ ->web,store",multi purchase,return
1,15,5968.82,1072.00,67.0,14.0,273.0,82.0,2022-02-09,2022-01-03,2023-10-07,...,2.0,only positive,mutli,2.0,1,"store, web",last web purchase,"return/ ->store,web",multi purchase,return
2,16,1656.04,35.79,53.0,1.0,118.0,2.0,2022-02-14,2023-02-07,2023-10-11,...,1.0,only positive,mono,1.0,1,"store, web",last store purchase,return/ web->store,multi purchase,return
3,19,5674.13,NaN,34.0,NaN,244.0,NaN,2022-05-05,NaT,2023-10-19,...,1.0,passive and positive,mutli,2.0,1,only store,only store,return,multi purchase,return
4,22,3663.47,NaN,32.0,NaN,36.0,NaN,2022-01-05,NaT,2023-07-20,...,1.0,passive and positive,mutli,3.0,1,only store,only store,return,multi purchase,return


### Summary by NPS

In [56]:
df_nps_summary_type = df_s_6.groupby('fb_nb_type')['user_id'].nunique().reset_index()
df_nps_summary_type['share_%'] = (df_nps_summary_type['user_id'] / df_nps_summary_type['user_id'].sum())*100
df_nps_summary_type

,fb_nb_type,user_id,share_%
0,mono,880891,72.439132
1,mutli,335152,27.560868


In [57]:
df_nps_summary_group = df_s_6.groupby('fb_group')['user_id'].nunique().reset_index()
df_nps_summary_group['share_%'] = (df_nps_summary_group['user_id'] / df_nps_summary_group['user_id'].sum())*100
df_nps_summary_group = df_nps_summary_group.sort_values(by='share_%', ascending=False)
df_nps_summary_group

,fb_group,user_id,share_%
5,only positive,675251,55.528546
4,only passive,275335,22.641880
3,only negative,128240,10.545680
6,passive and positive,77806,6.398293
1,negative and positive,26847,2.207734
0,negative and passive,21930,1.803390
2,"negative, passive, positive",10634,0.874476


In [58]:
df_nps_summary = df_s_6.groupby(['fb_nb_type', 'fb_group'])['user_id'].nunique().reset_index()
df_nps_summary['share_%'] = (df_nps_summary['user_id'] / df_nps_summary['user_id'].sum())*100
df_nps_summary = df_nps_summary.sort_values(by=['fb_nb_type','share_%'], ascending=[True, False])
df_nps_summary

,fb_nb_type,fb_group,user_id,share_%
2,mono,only positive,519465,42.717651
1,mono,only passive,244574,20.112282
0,mono,only negative,116852,9.609200
8,mutli,only positive,155786,12.810896
9,mutli,passive and positive,77806,6.398293
7,mutli,only passive,30761,2.529598
4,mutli,negative and positive,26847,2.207734
3,mutli,negative and passive,21930,1.803390
6,mutli,only negative,11388,0.936480
5,mutli,"negative, passive, positive",10634,0.874476


### Summary by Sales

In [70]:
df_s_nps_mm = df_s_6.groupby('purchase_nb')['user_id'].nunique().reset_index()
df_s_nps_mm.columns = ['purchase_nb', 'nb_users']
df_s_nps_mm['share_%'] = (df_s_nps_mm['nb_users'] / df_s_nps_mm['nb_users'].sum()*100).round(2)
df_s_nps_mm

,purchase_nb,nb_users,share_%
0,mono purchase,197891,16.27
1,multi purchase,1018171,83.73


In [71]:
df_s_nps_pt = df_s_6.groupby('purchase_type')['user_id'].nunique().reset_index()
df_s_nps_pt.columns = ['purchase_type', 'nb_users']
df_s_nps_pt['share_%'] = (df_s_nps_pt['nb_users'] / df_s_nps_pt['nb_users'].sum()*100).round(2)
df_s_nps_pt = df_s_nps_pt.sort_values(by='share_%', ascending=False)
df_s_nps_pt

,purchase_type,nb_users,share_%
2,"store, web",511487,42.06
0,only store,366016,30.10
1,only web,338559,27.84


In [73]:
df_s_6[(df_s_6['purchase_nb']=='multi purchase')]['purchase_type'].unique()

array(['store, web', 'only store', 'only web'], dtype=object)

In [74]:
df_s_nps_mm_t = df_s_6.groupby(['purchase_nb', 'purchase_type'])['user_id'].nunique().reset_index()
df_s_nps_mm_t.columns = ['purchase_nb', 'purchase_type', 'nb_users']
df_s_nps_mm_t['share_%'] = (df_s_nps_mm_t['nb_users'] / df_s_nps_mm_t['nb_users'].sum()*100).round(2)
df_s_nps_mm_t = df_s_nps_mm_t.sort_values(by='share_%', ascending=False)
df_s_nps_mm_t

,purchase_nb,purchase_type,nb_users,share_%
4,multi purchase,"store, web",511487,42.06
2,multi purchase,only store,340042,27.96
1,mono purchase,only web,171917,14.14
3,multi purchase,only web,166642,13.70
0,mono purchase,only store,25974,2.14


In [79]:
df_s_nps_ret_gr = df_s_6.groupby('return_gr')['user_id'].nunique().reset_index()
df_s_nps_ret_gr.columns = ['return_flag', 'nb_users']
df_s_nps_ret_gr['share_%'] = (df_s_nps_ret_gr['nb_users'] / df_s_nps_ret_gr['nb_users'].sum()*100).round(2)
df_s_nps_ret_gr = df_s_nps_ret_gr.sort_values(by='share_%', ascending=False)
df_s_nps_ret_gr

,return_flag,nb_users,share_%
1,return,973366,80.04
0,no return,242696,19.96


In [81]:
df_s_nps_ret = df_s_6.groupby('return_flag')['user_id'].nunique().reset_index()
df_s_nps_ret.columns = ['return_flag', 'nb_users']
df_s_nps_ret['share_%'] = (df_s_nps_ret['nb_users'] / df_s_nps_ret['nb_users'].sum()*100).round(2)
df_s_nps_ret = df_s_nps_ret.sort_values(by='return_flag', ascending=True)
df_s_nps_ret

,return_flag,nb_users,share_%
0,no return,242696,19.96
1,return,505064,41.53
2,"return/ ->store,web",52322,4.30
3,"return/ ->web,store",239121,19.66
4,return/ store->web,45526,3.74
5,return/ web->store,131333,10.80


In [84]:
df_s_nps_lp_1 = df_s_6.groupby(['purchase_type', 'last_purchase'])['user_id'].nunique().reset_index()
df_s_nps_lp_1.columns = ['purchase_type', 'last_purchase', 'nb_users']
df_s_nps_lp_1['share_%'] = (df_s_nps_lp_1['nb_users'] / df_s_nps_lp_1['nb_users'].sum()*100).round(2)
df_s_nps_lp_1 = df_s_nps_lp_1.sort_values(by='share_%', ascending=False)
df_s_nps_lp_1

,purchase_type,last_purchase,nb_users,share_%
2,"store, web",last store purchase,392600,32.28
0,only store,only store,366016,30.10
1,only web,only web,338559,27.84
3,"store, web",last web purchase,118887,9.78


In [88]:
df_s_nps_ret2 = df_s_6.groupby(['return_gr','purchase_type'])['user_id'].nunique().reset_index()
df_s_nps_ret2.columns = ['return_gr', 'purchase_type', 'nb_users']
df_s_nps_ret2['share_%'] = (df_s_nps_ret2['nb_users'] / df_s_nps_ret2['nb_users'].sum()*100).round(2)
df_s_nps_ret2 = df_s_nps_ret2.sort_values(by='share_%', ascending=False)
df_s_nps_ret2

,return_gr,purchase_type,nb_users,share_%
5,return,"store, web",468302,38.51
3,return,only store,288526,23.73
4,return,only web,216538,17.81
1,no return,only web,122021,10.03
0,no return,only store,77490,6.37
2,no return,"store, web",43185,3.55


In [90]:
df_s_nps_lp = df_s_6.groupby(['purchase_nb', 'purchase_type', 'last_purchase'])['user_id'].nunique().reset_index()
df_s_nps_lp.columns = ['purchase_nb', 'purchase_type', 'last_purchase', 'nb_users']
df_s_nps_lp['share_%'] = (df_s_nps_lp['nb_users'] / df_s_nps_lp['nb_users'].sum()*100).round(2)
df_s_nps_lp = df_s_nps_lp.sort_values(by='share_%', ascending=False)
df_s_nps_lp

,purchase_nb,purchase_type,last_purchase,nb_users,share_%
4,multi purchase,"store, web",last store purchase,392600,32.28
2,multi purchase,only store,only store,340042,27.96
1,mono purchase,only web,only web,171917,14.14
3,multi purchase,only web,only web,166642,13.70
5,multi purchase,"store, web",last web purchase,118887,9.78
0,mono purchase,only store,only store,25974,2.14


In [91]:
df_s_nps_lp3 = df_s_6.groupby(['return_gr','purchase_nb', 'purchase_type', 'last_purchase'])['user_id'].nunique().reset_index()
df_s_nps_lp3.columns = ['return_gr','purchase_nb', 'purchase_type', 'last_purchase', 'nb_users']
df_s_nps_lp3['share_%'] = (df_s_nps_lp3['nb_users'] / df_s_nps_lp3['nb_users'].sum()*100).round(2)
df_s_nps_lp3 = df_s_nps_lp3.sort_values(by='share_%', ascending=False)
df_s_nps_lp3

,return_gr,purchase_nb,purchase_type,last_purchase,nb_users,share_%
10,return,multi purchase,"store, web",last store purchase,370454,30.46
8,return,multi purchase,only store,only store,287829,23.67
9,return,multi purchase,only web,only web,130061,10.70
11,return,multi purchase,"store, web",last web purchase,97848,8.05
7,return,mono purchase,only web,only web,86477,7.11
1,no return,mono purchase,only web,only web,85440,7.03
2,no return,multi purchase,only store,only store,52213,4.29
3,no return,multi purchase,only web,only web,36581,3.01
0,no return,mono purchase,only store,only store,25277,2.08
4,no return,multi purchase,"store, web",last store purchase,22146,1.82


### Summary Total

In [101]:
df_s_nps_lp4 = df_s_6.groupby(['return_gr','purchase_nb',  'fb_nb_type'])['user_id'].nunique().reset_index()
df_s_nps_lp4.columns = ['return_gr','purchase_nb',  'fb_nb_type', 'nb_users']
df_s_nps_lp4['share_%'] = (df_s_nps_lp4['nb_users'] / df_s_nps_lp4['nb_users'].sum()*100).round(2)
df_s_nps_lp4 = df_s_nps_lp4.sort_values(by=['return_gr','share_%'], ascending = [True, False])
df_s_nps_lp4

,return_gr,purchase_nb,fb_nb_type,nb_users,share_%
0,no return,mono purchase,mono,107129,8.81
2,no return,multi purchase,mono,82256,6.76
3,no return,multi purchase,mutli,49723,4.09
1,no return,mono purchase,mutli,3588,0.30
6,return,multi purchase,mono,604634,49.72
7,return,multi purchase,mutli,281539,23.15
4,return,mono purchase,mono,86872,7.14
5,return,mono purchase,mutli,302,0.02


In [98]:
df_s_nps_gr = df_s_6.groupby(['return_gr','return_flag', 'purchase_nb', 'purchase_type','last_purchase' , 'fb_nb_type'])\
                        ['user_id'].nunique().reset_index()
df_s_nps_gr.columns = ['return_gr','return_flag', 'purchase_nb', 'purchase_type','last_purchase' , 'fb_nb_type','nb_users']
df_s_nps_gr['share_%'] = (df_s_nps_gr['nb_users'] / df_s_nps_gr['nb_users'].sum()*100).round(2)
df_s_nps_gr = df_s_nps_gr.sort_values(by=['return_gr','share_%'], ascending = [True, False])
df_s_nps_gr

,return_gr,return_flag,purchase_nb,purchase_type,last_purchase,fb_nb_type,nb_users,share_%
2,no return,no return,mono purchase,only web,only web,mono,82421,6.78
4,no return,no return,multi purchase,only store,only store,mono,32137,2.64
6,no return,no return,multi purchase,only web,only web,mono,24911,2.05
0,no return,no return,mono purchase,only store,only store,mono,24708,2.03
5,no return,no return,multi purchase,only store,only store,mutli,20076,1.65
10,no return,no return,multi purchase,"store, web",last web purchase,mono,13586,1.12
7,no return,no return,multi purchase,only web,only web,mutli,11670,0.96
8,no return,no return,multi purchase,"store, web",last store purchase,mono,11622,0.96
9,no return,no return,multi purchase,"store, web",last store purchase,mutli,10524,0.87
11,no return,no return,multi purchase,"store, web",last web purchase,mutli,7453,0.61


In [103]:
df_s_nps_res = df_s_6.groupby(['return_flag', 'return_gr', 'purchase_type','purchase_nb', 'last_purchase', 'fb_nb_type', 'fb_group'])\
                        ['user_id'].nunique().reset_index()
df_s_nps_res.columns = ['return_flag', 'return_gr', 'purchase_type','purchase_nb', 'last_purchase', 'fb_nb_type', 'fb_group', 'nb_users']
df_s_nps_res['share_%'] = (df_s_nps_res['nb_users'] / df_s_nps_res['nb_users'].sum()*100).round(2)
df_s_nps_res

,return_flag,return_gr,purchase_type,purchase_nb,last_purchase,fb_nb_type,fb_group,nb_users,share_%
0,no return,no return,only store,mono purchase,only store,mono,only negative,3464,0.28
1,no return,no return,only store,mono purchase,only store,mono,only passive,6242,0.51
2,no return,no return,only store,mono purchase,only store,mono,only positive,15002,1.23
3,no return,no return,only store,mono purchase,only store,mutli,negative and passive,42,0.00
4,no return,no return,only store,mono purchase,only store,mutli,negative and positive,86,0.01
...,...,...,...,...,...,...,...,...,...
129,return/ web->store,return,"store, web",multi purchase,last store purchase,mutli,"negative, passive, positive",2637,0.22
130,return/ web->store,return,"store, web",multi purchase,last store purchase,mutli,only negative,1870,0.15
131,return/ web->store,return,"store, web",multi purchase,last store purchase,mutli,only passive,4392,0.36
132,return/ web->store,return,"store, web",multi purchase,last store purchase,mutli,only positive,26002,2.14


In [104]:
df_s_nps_res[df_s_nps_res['return_flag']=='no return'].sort_values(by='share_%', ascending=False)

,return_flag,return_gr,purchase_type,purchase_nb,last_purchase,fb_nb_type,fb_group,nb_users,share_%
22,no return,no return,only web,mono purchase,only web,mono,only positive,48854,4.02
21,no return,no return,only web,mono purchase,only web,mono,only passive,25413,2.09
12,no return,no return,only store,multi purchase,only store,mono,only positive,18871,1.55
32,no return,no return,only web,multi purchase,only web,mono,only positive,15182,1.25
2,no return,no return,only store,mono purchase,only store,mono,only positive,15002,1.23
18,no return,no return,only store,multi purchase,only store,mutli,only positive,9346,0.77
52,no return,no return,"store, web",multi purchase,last web purchase,mono,only positive,8387,0.69
20,no return,no return,only web,mono purchase,only web,mono,only negative,8154,0.67
11,no return,no return,only store,multi purchase,only store,mono,only passive,7733,0.64
42,no return,no return,"store, web",multi purchase,last store purchase,mono,only positive,6893,0.57
